# <a id='toc1_'></a>[**Automating Port Operations**](#toc0_)

-----------------------------
## <a id='toc1_1_'></a>[**Context**](#toc0_)
-----------------------------

Marina Pier Inc. is leveraging technology to automate their operations on the San Francisco port.
The company’s management has set out to build a bias-free/ corruption-free automatic system that reports & avoids faulty situations caused by human error. 

-----------------------------
## <a id='toc1_2_'></a>[**Objectives**](#toc0_)
-----------------------------

Marina Pier wants to use Deep Learning techniques to build an automatic reporting system that recognizes the boat. The company is also looking to use a transfer learning approach of any lightweight pre-trained model in order to deploy in mobile devices.
As a deep learning engineer, your task is to:

1.	Build a CNN network to classify the boat.

2.	Build a lightweight model with the aim of deploying the solution on a mobile device using transfer learning. You can use any lightweight pre-trained model as the initial (first) layer. MobileNetV2 is a popular lightweight pre-trained model built using Keras API. 

-----------------------------
## <a id='toc1_3_'></a>[**Dataset**](#toc0_)
-----------------------------

**boat_type_classification_dataset.zip**

The dataset contains images of 9 types of boats. It contains a total of 1162 images. The training images are provided in the directory of the specific class itself. 
Classes:

- ferry_boat
- gondola
- sailboat
- cruise_ship
- kayak
- inflatable_boat
- paper_boat
- buoy
- freight_boat


-----------------------------------
## <a id='toc1_4_'></a>[**Analysis Steps to Perform**](#toc0_)
-----------------------------------
1.	Build a CNN network to classify the boat.

    1.1.	Split the dataset into train and test in the ratio 80:20, with shuffle and random state=43. 

    1.2.	Use tf.keras.preprocessing.image_dataset_from_directory to load the train and test datasets. This function also supports data normalization.*(Hint: image_scale=1./255)*

    1.3.	Load train, validation and test dataset in batches of 32 using the function initialized in the above step. 

    1.4.	Build a CNN network using Keras with the following layers

       - Cov2D with 32 filters, kernel size 3,3, and activation relu, followed by MaxPool2D

       - Cov2D with 32 filters, kernel size 3,3, and activation relu, followed by MaxPool2D

       - GLobalAveragePooling2D layer

       - Dense layer with 128 neurons and activation relu

       - Dense layer with 128 neurons and activation relu
    
       - Dense layer with 9 neurons and activation softmax.

    1.5.	Compile the model with Adam optimizer, categorical_crossentropy loss, and with metrics accuracy, precision, and recall.

    1.6.	Train the model for 20 epochs and plot training loss and accuracy against epochs.

    1.7.	Evaluate the model on test images and print the test loss and accuracy.

    1.8.	Plot heatmap of the confusion matrix and print classification report.

2.	Build a lightweight model with the aim of deploying the solution on a mobile device using transfer learning. You can use any lightweight pre-trained model as the initial (first) layer. MobileNetV2 is a popular lightweight pre-trained model built using Keras API. 

    2.1.	Split the dataset into train and test datasets in the ration 70:30, with shuffle and random state=1.

    2.2.	Use tf.keras.preprocessing.image_dataset_from_directory to load the train and test datasets. This function also supports data normalization.*(Hint: Image_scale=1./255)*

    2.3.	Load train, validation and test datasets in batches of 32 using the function initialized in the above step.

    2.4.	Build a CNN network using Keras with the following layers. 

      - Load MobileNetV2 - Light Model as the first layer *(Hint: Keras API Doc)*

      - GLobalAveragePooling2D layer

      - Dropout(0.2)

      - Dense layer with 256 neurons and activation relu

      - BatchNormalization layer

      - Dropout(0.1)

      - Dense layer with 128 neurons and activation relu

      - BatchNormalization layer

      - Dropout(0.1)

      - Dense layer with 9 neurons and activation softmax

    2.5.	Compile the model with Adam optimizer, categorical_crossentropy loss, and metrics accuracy, Precision, and Recall.

    2.6.	Train the model for 50 epochs and Early stopping while monitoring validation loss.

    2.7.	Evaluate the model on test images and print the test loss and accuracy.

    2.8.	Plot Train loss Vs Validation loss and Train accuracy Vs Validation accuracy.
    
3.	Compare the results of both models built in steps 1 and 2 and state your observations.



### <a id='toc1_4_1_'></a>[**Setup: Import Necessary Libraries**](#toc0_)

In [ ]:
import pathlib
import os
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

### <a id='toc1_4_2_'></a>[**1. Build A CNN Network To Classify A Boat**](#toc0_)

#### <a id='toc1_4_2_1'></a>[**1.1 Split the dataset into train and test in the ratio 80:20, with shuffle and random state=43**](#toc0_)

In [ ]:
# Set the path to your dataset
data_dir = pathlib.Path("boat_type_classification_dataset")

# Set common parameters
batch_size = 32
img_height = 224
img_width = 224

# Load and split the full dataset
full_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="both",
    seed=43,
    image_size=(img_height, img_width),
    batch_size=None,  # Load without batching initially
    shuffle=True
)

# Split the full dataset into train and test
train_ds, test_ds = full_ds

print("Number of training samples:", tf.data.experimental.cardinality(train_ds))
print("Number of test samples:", tf.data.experimental.cardinality(test_ds))

**Explanation:**

- This section is responsible for loading the dataset and splitting it into training and testing sets.
- We use `tf.keras.preprocessing.image_dataset_from_directory` to load images directly from the filesystem. This function is convenient as it handles the file reading and label assignment automatically.
- We set `validation_split=0.2` and `subset="both"` to get both training and testing sets in an 80:20 ratio.
- `seed=43` ensures reproducibility of the random split.
- `shuffle=True` randomizes the order of the samples, which is important for training neural networks.
- We set `batch_size=None` initially to load the entire dataset without batching, giving us more flexibility in the subsequent processing steps.
- The `image_size` parameter resizes all images to a consistent size, which is necessary for batch processing in neural networks.

**Why it's important:**

- Properly splitting the data ensures we have separate sets for training and evaluation, which is crucial for assessing the model's performance on unseen data.
- The 80:20 split is a common ratio that balances having enough training data while still retaining a significant portion for testing.
- Shuffling the data helps prevent any bias that might occur from the order of the samples.

#### <a id='toc1_4_2_2'></a>[**1.2 Use tf.keras.preprocessing.image_dataset_from_directory to load the train and test datasets. This function also supports data normalization.** *(Hint: image_scale=1./255)*](#toc0_)

In [ ]:
def normalize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    return tf.cast(image, tf.float32) / 255., label

# Apply normalization to the datasets
train_ds = train_ds.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)

# Function to safely check normalization
def check_normalization(dataset):
    for images, _ in dataset.take(1):
        min_val = tf.reduce_min(images).numpy()
        max_val = tf.reduce_max(images).numpy()
        return min_val, max_val
    return None, None

print("Checking train dataset normalization:")
train_min, train_max = check_normalization(train_ds)
if train_min is not None and train_max is not None:
    print(f"Image data range: {train_min} to {train_max}")
else:
    print("Unable to check train dataset normalization")

print("\nChecking test dataset normalization:")
test_min, test_max = check_normalization(test_ds)
if test_min is not None and test_max is not None:
    print(f"Image data range: {test_min} to {test_max}")
else:
    print("Unable to check test dataset normalization")

**Explanation:**

- This section focuses on normalizing the image data and verifying that the normalization was applied correctly.
- We define a `normalize_img` function that converts the image data from integers in the range [0, 255] to floating-point numbers in the range [0, 1].
- We apply this normalization to both the training and testing datasets using the map function.
- The `check_normalization` function safely checks the range of values in the normalized datasets.

**Why it's important:**

- Normalization is crucial for neural network training. It helps the model converge faster and can lead to better performance.
- Scaling the input to a standard range (like [0, 1]) ensures that all features contribute equally to the model's learning process.
- Checking the normalization helps verify that our preprocessing steps are working as expected.

#### <a id='toc1_4_2_3'></a>[**1.3 Load train, validation and test dataset in batches of 32 using the function initialized in the above step.**](#toc0_)

In [ ]:
# Function to prepare dataset
def prepare_dataset(dataset, is_training=False):
    # Shuffle the dataset if it's the training set
    if is_training:
        dataset = dataset.shuffle(buffer_size=1000)
    
    # Batch the dataset
    dataset = dataset.batch(batch_size)
    
    # Use data augmentation only on the training set
    if is_training:
        data_augmentation = tf.keras.Sequential([
            tf.keras.layers.RandomFlip('horizontal'),
            tf.keras.layers.RandomRotation(0.2),
        ])
        dataset = dataset.map(lambda x, y: (data_augmentation(x, training=True), y),
                              num_parallel_calls=tf.data.AUTOTUNE)
    
    # Use buffered prefetching on all datasets
    return dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

# Prepare datasets
train_ds = prepare_dataset(train_ds, is_training=True)
test_ds = prepare_dataset(test_ds)

# Further split test_ds into validation and test
val_ds = test_ds.take(tf.data.experimental.cardinality(test_ds) // 2)
test_ds = test_ds.skip(tf.data.experimental.cardinality(test_ds) // 2)

# Print dataset information
print("Number of training batches:", tf.data.experimental.cardinality(train_ds))
print("Number of validation batches:", tf.data.experimental.cardinality(val_ds))
print("Number of test batches:", tf.data.experimental.cardinality(test_ds))

# Function to safely get a batch from a dataset
def get_batch(dataset):
    for batch in dataset.take(1):
        return batch
    print("Dataset is empty")
    return None

# Print an example batch to verify the shape
example_batch = get_batch(train_ds)
if example_batch is not None:
    images, labels = example_batch
    print("Image batch shape:", images.shape)
    print("Label batch shape:", labels.shape)

# Get class names
class_names = full_ds[0].class_names
print("Class names:", class_names)

**Explanation:**

- This section prepares the datasets for training by applying several important transformations.
- The prepare_dataset function:
    - Shuffles the training data to randomize the order of samples in each epoch.
    - Batches the data into groups of 32 images (as specified by batch_size).
    - Applies data augmentation (random flips and rotations) to the training set only.
    - Uses prefetching to optimize data loading performance.
- We split the test set further into validation and test sets.
- We print information about the datasets and verify the shape of a sample batch.

**Why it's important**:

- Batching is necessary for efficient processing in neural networks. It allows the model to update its weights based on multiple samples at once.
- Data augmentation helps prevent overfitting by artificially increasing the diversity of the training set.
- Shuffling the training data ensures that the model sees the data in a different order each epoch, which can improve generalization.
- Prefetching helps optimize performance by preparing the next batch of data while the current batch is being processed.
- Splitting the test set into validation and test sets allows us to tune hyperparameters on the validation set while still having a completely unseen test set for final evaluation.
- Verifying the shapes of the batches ensures that our data is in the correct format for our model.